In [1]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [2]:
#parameters()
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = datasets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = datasets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [12]:
#model
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        #L1 image shape batch 28 28 1 
        #L2 image shape  btach 28 28 32
        # Pool -> batch  14 14 32
        self.layer1=torch.nn.Sequential(
                torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),  # N-k+2p/s+1  28-3+2/1+1 28
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2,stride=2)  #28-2/2+1  padding=0 
        )
        self.layer2=nn.Sequential(
                torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2,stride=2) # 7x7 x64
        )
        self.fc=torch.nn.Linear(7*7*64,10,bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out) #(batch_size ,64x64x7) (-1, 64x64x7)
        out=out.view(out.size(0),-1)
        out=self.fc(out)
        return out
        

In [13]:
# instantiate CNN model
model = CNN().to(device)
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# train my model
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.232505664
[Epoch:    2] cost = 0.0636391565
[Epoch:    3] cost = 0.0487953536
[Epoch:    4] cost = 0.0400274061
[Epoch:    5] cost = 0.0330952443
[Epoch:    6] cost = 0.0283723995
[Epoch:    7] cost = 0.0241516493
[Epoch:    8] cost = 0.020737838
[Epoch:    9] cost = 0.0180728938
[Epoch:   10] cost = 0.0150964875
[Epoch:   11] cost = 0.0124662276
[Epoch:   12] cost = 0.0111330226
[Epoch:   13] cost = 0.00957726501
[Epoch:   14] cost = 0.00724629126
[Epoch:   15] cost = 0.00840050355
Learning Finished!


In [14]:
# Test model and check accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9896000027656555


/root/miniconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/root/miniconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [ ]:


import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random
import warnings
warnings.filterwarnings("ignore")

# 种子
random.seed(1)
torch.manual_seed(1)
device='cuda' if torch.cuda.is_available() else 'cpu'
if device=='cuda':
    torch.cuda.manual_seed_all(1)

#设置
epochs=20
batch_size=100
# 数据
trains=datasets.MNIST(root='data',train=True,download=True,transform=transforms.ToTensor())
tests=datasets.MNIST(root='data',train=False,download=True,transform=transforms.ToTensor())

# 打包
data_loader=torch.utils.data.DataLoader(dataset=trains,batch_size=batch_size,drop_last=True,shuffle=True)
# model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # batch_size 28x28x1
        self.layer1=nn.Sequential(nn.Conv2d(1,32,stride=1,padding=1,kernel_size=3), # 28-3+2/1+1
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2,padding=0,stride=2)  #28-2/2+1
                                  )
        self.layer2=nn.Sequential(nn.Conv2d(32,64,stride=1,padding=1,kernel_size=3),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2,padding=0,stride=2)
                                  )
        self.fc=nn.Linear(7*7*64,10,bias=True)
        torch.nn.init.xavier_normal_(self.fc.weight)
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(x)
        out = out.view(out.size(0), -1)
        out=self.fc(out)
        return out
model=CNN().to(device)
# 优化器
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(),lr=0.1)
model.train()
total_batch=len(data_loader)
for step in range(epochs+1):
    avg_loss=0
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        Y_pred=model(X)
        loss=criterion(Y_pred,Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss=loss/total_batch
    print("[epochs: {:>4}], loss= {:>6}".format(epochs,avg_loss.item()))

model.eval()
with torch.no_grad():
    X=tests.test_data.view(len(tests),1,28,28).to(device)
    Y=tests.train_labels.to(device)
    Y_pred=model(X)
    # corr=(Y_pred>=0.5).float()
    acc=(torch.argmax(Y_pred,1)==Y).float().mean()
    # acc= (corr==Y).float().mean()
    print(acc)

In [1]:
#CNN
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random
import warnings
warnings.filterwarnings('ignore')
# 
random.seed(1)
torch.manual_seed(1)
device='cuda' if torch.cuda.is_available() else 'cpu'
if device=='cuda':
    torch.cuda.manual_seed_all(7)

In [2]:
# download data
trains=datasets.MNIST(root='data',train=True,transform=transforms.ToTensor(),download=True)
tests=datasets.MNIST(root='data',train=False,transform=transforms.ToTensor(),download=True)

In [3]:
# dataloader
data_loader=torch.utils.data.DataLoader(dataset=trains,shuffle=True,drop_last=True,batch_size=1000)

In [7]:
next(iter(data_loader))[0].shape

torch.Size([1000, 1, 28, 28])

In [8]:
next(iter(data_loader))[1].shape # y

torch.Size([1000])

In [16]:
#model
class Cnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),           # 28-3+2p/1+1  ? 32 28*28
            nn.ReLU(),
            nn.MaxPool2d(stride=2,padding=0,kernel_size=2)  # 28-k/2+1 14   ? 32 14*14
        )
        self.layer2=nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2,padding=0)  #?  64 7*7 14-2/2+1   ? 64, 7,7
        )
        self.fc1=nn.Linear(64*7*7,128,bias=True)
        self.fc2=nn.Linear(128,10,bias=True)
        torch.nn.init.xavier_normal_(self.fc1.weight)
        torch.nn.init.xavier_normal_(self.fc2.weight)
        self.relu=nn.ReLU()
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)#? 64, 7,7
        out=out.view(-1,64*7*7)
        out=self.fc1(out)
        out=self.relu(out)
        out=self.fc2(out)
        return out

In [17]:
model=Cnn().to(device)

In [18]:
model

Cnn(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
)

In [19]:
# 
criterion=nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [20]:
total_batch=len(data_loader)
epochs=20
model.train()
for step in range(epochs):
    avg_loss=0
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        Y_pred=model(X)
        loss=criterion(Y_pred,Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss=loss/total_batch
    print("[epcohs: {:>4}] loss= {:>7}".format(step,avg_loss))

[epcohs:    0] loss= 0.001648039324209094
[epcohs:    1] loss= 0.0010754335671663284
[epcohs:    2] loss= 0.000914833159185946
[epcohs:    3] loss= 0.0007333779940381646
[epcohs:    4] loss= 0.000665927364025265
[epcohs:    5] loss= 0.00047612254275009036
[epcohs:    6] loss= 0.00027294052415527403
[epcohs:    7] loss= 0.000451659900136292
[epcohs:    8] loss= 0.0004184455028735101
[epcohs:    9] loss= 0.00035330469836480916
[epcohs:   10] loss= 0.00015084743790794164
[epcohs:   11] loss= 0.00019414607959333807
[epcohs:   12] loss= 0.00018826115410774946
[epcohs:   13] loss= 0.00019428365339990705
[epcohs:   14] loss= 0.00010645194561220706
[epcohs:   15] loss= 0.0001480443315813318
[epcohs:   16] loss= 0.00024270740686915815
[epcohs:   17] loss= 0.00020266477076802403
[epcohs:   18] loss= 0.00016788770153652877
[epcohs:   19] loss= 4.534313848125748e-05


In [21]:
next(iter(tests))[0].shape

torch.Size([1, 28, 28])

In [22]:
tests.test_data.shape

torch.Size([10000, 28, 28])

In [23]:
len(tests)

10000

In [25]:
batch_test=len(tests)
model.eval()
with torch.no_grad():
    X_=tests.test_data.view(batch_test,1,28,28).float().to(device)
    Y_=tests.test_labels.to(device)
    Y_pred=model(X_)
    acc=(torch.argmax(Y_pred,1)==Y_).float().mean()
    print(acc)

tensor(0.9864, device='cuda:0')


In [26]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


In [27]:
vis.text("Hello, world!",env="main")

'window_3bdf56127ec704'

In [28]:
vis.text("Hello, world!",env="main")

'window_3bdf56165ea270'